In [13]:
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anjim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
DATA_PATH = 'data'
MIMIC_PATH = 'E:\\codes\\mimic-iii-clinical-database-1.4'
EMBED_PATH = 'embedding'
# CHECKPOINT_PATH_ms_model1 = 'checkpoints/ms_model1'

In [7]:
# create cohort -- will print the cohort statistics during processing
!python create_cohorts.py --data_dir $DATA_PATH --mimic_dir $MIMIC_PATH --threshold 48

Construct cohort for ms drg
raw cases w/ MS drg: pt - 21551, stays - 27267
age over 18 w/ drg: pt - 20867, visits - 26567
plus criterion on single icu: pt - 17976, visits - 21938
Loading MIMIC notes...209iter
565455 note events in total...
477048 note events w/ charttime
at least one note before the 48h threshold at icu
..there are 16484 pt, 19792 stays w/ ms drg in total
..split into train (14836 pt, 17809 st), and test (1648 pt, 1983 st).




Construct cohort for apr drg
40493 raw apr codes in MIMIC, 39542 after first dropping duplicated severity
raw cases w/ APR drg: pt - 31085, stays - 39248
age over 18 w/ drg: pt - 27767, visits - 35845
plus criterion on single icu: pt - 23979, visits - 29596
Loading MIMIC notes...209iter
734255 note events in total...
614512 note events w/ charttime
at least one note before the 48h threshold at icu
..there are 22518 pt, 27414 stays w/ apr drg in total
..split into train (20266 pt, 24667 st), and test (2252 pt, 2747 st).







100%|██████████| 209/209 [00:39<00:00,  5.29it/s]

 42%|████▏     | 9137/21938 [01:31<02:10, 98.23it/s]


100%|██████████| 21938/21938 [03:29<00:00, 104.58it/s]

100%|██████████| 209/209 [00:40<00:00,  5.21it/s]

100%|██████████| 29596/29596 [05:23<00:00, 91.62it/s]


In [ ]:
# preprocessing text for MS-DRG cohort 
!python preprocessing.py --data_dir $DATA_PATH --pretrained_embed_dir $EMBED_PATH --word_min_freq 3 --cohort ms

39386136 tokens in text, 241276 unique, and 82086 of them appeared at least three times
loading biovec...
loaded, start to get embed for tokens



 12%|█▏        | 2290/19792 [03:36<14:03, 20.74it/s]


100%|██████████| 19792/19792 [25:44<00:00, 12.81it/s]
Traceback (most recent call last):
  File "preprocessing.py", line 221, in <module>
    main()
  File "preprocessing.py", line 214, in main
    token2id = get_embeddings(words, output_dir)
  File "preprocessing.py", line 156, in get_embeddings
    model_vocab = set(model.index2word)
  File "c:\Users\anjim\anaconda3\envs\LLM_SALUD\lib\site-packages\gensim\models\keyedvectors.py", line 725, in index2word
    "The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.\n"
AttributeError: The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4


In [22]:
# run CAML on the MS-DRG cohort
!python main.py --epochs 50 --patience 10 --max_seq_length 2000 --model CAML --single_kernel_size 5 --lr 1e-4 --wd 0 --cohort ms --device 0 --data_dir $DATA_PATH --pretrained_embed_dir $EMBED_PATH


Traceback (most recent call last):
  File "main.py", line 42, in <module>
    train_dataset = DrgTextDataset(args, train_df, RULE_PATH)
  File "e:\codes\LLM_CASO_SALUD\data_utils.py", line 23, in __init__
    self.load_data(48)
  File "e:\codes\LLM_CASO_SALUD\data_utils.py", line 29, in load_data
    self.data = self.read_df(self.df, hour)
  File "e:\codes\LLM_CASO_SALUD\data_utils.py", line 53, in read_df
    X_text = self.read_text(stay, hour)
  File "e:\codes\LLM_CASO_SALUD\data_utils.py", line 65, in read_text
    with open(path, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'data/ms/text_embed/5689_157267.dict'
